In [0]:
# imports
import re
import pandas as pd

# open csv file generated from database(big) and chose words used more than 5 times as refrence
df = pd.read_csv('output big.csv')
df = df.drop(['DF', '1 TF-IDF'], axis=1)
df = df[df['1 TF'] > 5]
DICTIONARY = df.set_index('Word')['1 TF'].to_dict()
SUM = sum(DICTIONARY.values())

In [0]:
def probability(newWord, originalWord):
  bias = 1
  if len(newWord) == len(originalWord):
    bias = 1.1
  elif len(originalWord) - 1 < len(newWord) < len(originalWord) + 1:
    bias = 1.05
  return DICTIONARY[newWord] / SUM * bias

def known(words, word):
    "The subset of `words` that appear in the dictionary of WORDS."
    return {w: probability(w, word) for w in words if w in DICTIONARY}

def edits1(word):
    # All edits that are one edit away from 'word'
    letters    = 'abcdefghijklmnopqrstuvwxyzءآابپتثجچحخدذرزسشصضطظعغفقکگلمنوهی'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def closestWords(word):
    # if word in dictionary return it
    if(word in DICTIONARY):
      return {word:1}
    # If one edit avalible return it else return two edits away from `word`.
    edits = edits1(word)
    known1 = known(edits, word)
    if len(known1) != 0:
      return known1
    if len(word) == 2:
      return {}
    editss = (e2 for e1 in edits1(word) for e2 in edits1(e1))
    return known(editss, word)

In [124]:
text = input('Enter the word ')
for word in text.split(' '):
  correct = closestWords(word)
  if len(correct)!=0:
    print(f'{word} is likely {max(correct, key=correct.get)}')
    print(correct)
  else:
    print(f'No match found for {word}')

Enter the word سلام سلامتی چطو تس اتمکهدو
سلام is likely سلام
{'سلام': 1}
سلامتی is likely سلامتی
{'سلامتی': 1}
چطو is likely چطور
{'چطور': 6.723750630351622e-05}
تس is likely تا
{'تز': 2.129187699611347e-05, 'تو': 0.00025102002353312723, 'ت': 1.5281251432617322e-05, 'تک': 0.00012887188708173944, 'تب': 0.00010197688456033294, 'ست': 2.5774377416347887e-05, 'تن': 0.00038213316082498386, 'تی': 1.5688751470820452e-05, 'س': 0.000199675018719533, 'اس': 3.8101253571992525e-05, 'کس': 0.00014007813813232545, 'تست': 2.2412502101172073e-05, 'حس': 2.689500252140649e-05, 'ترس': 3.973125372480504e-05, 'پس': 0.0011990688624127059, 'توس': 1.42625013371095e-05, 'تا': 0.0026939827525608835, 'بس': 2.689500252140649e-05}
No match found for اتمکهدو
